# Introduction

In this guide, we will walk you through building a powerful semantic search engine using Couchbase as the backend database and [CrewAI](https://github.com/joaomdmoura/crewAI) for agent-based RAG operations. CrewAI allows us to create specialized agents that can work together to handle different aspects of the RAG workflow, from document retrieval to response generation. This tutorial is designed to be beginner-friendly, with clear, step-by-step instructions that will equip you with the knowledge to create a fully functional semantic search system from scratch.

# Setting the Stage: Installing Necessary Libraries

To build our semantic search engine, we need a robust set of tools. The libraries we install handle everything from connecting to databases to performing complex machine learning tasks.

In [ ]:
%pip install datasets langchain-couchbase langchain-openai crewai python-dotenv

# Importing Necessary Libraries

The script starts by importing a series of libraries required for various tasks, including handling JSON, logging, time tracking, Couchbase connections, embedding generation, and dataset loading.

In [ ]:
import json
import logging
import time
import sys
import os
from datetime import timedelta
from uuid import uuid4
from dotenv import load_dotenv

from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.exceptions import CouchbaseException, InternalServerFailureException, QueryIndexAlreadyExistsException
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from datasets import load_dataset
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.documents import Document
from langchain_core.globals import set_llm_cache
from langchain_core.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_couchbase.cache import CouchbaseCache
from langchain_couchbase.vectorstores import CouchbaseVectorStore
from crewai import Agent, Task, Crew, Process
from tqdm import tqdm

# Setup Logging

Logging is configured to track the progress of the script and capture any errors or warnings.

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

# Loading Configuration Settings

In this section, we load configuration settings from environment variables or prompt the user for input. These settings include API keys, database credentials, and specific configuration names.

In [ ]:
import getpass

# Load environment variables from .env file if it exists
load_dotenv()

# OpenAI API Key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or getpass.getpass('Enter your OpenAI API key: ')

# Couchbase Settings
CB_HOST = os.getenv('CB_HOST') or input('Enter your Couchbase host (default: couchbase://localhost): ') or 'couchbase://localhost'
CB_USERNAME = os.getenv('CB_USERNAME') or input('Enter your Couchbase username (default: Administrator): ') or 'Administrator'
CB_PASSWORD = os.getenv('CB_PASSWORD') or getpass.getpass('Enter your Couchbase password (default: password): ') or 'password'
CB_BUCKET_NAME = os.getenv('CB_BUCKET_NAME') or input('Enter your Couchbase bucket name (default: vector-search-testing): ') or 'vector-search-testing'
INDEX_NAME = input('Enter your index name (default: vector_search_crew): ') or 'vector_search_crew'
SCOPE_NAME = input('Enter your scope name (default: shared): ') or 'shared'
COLLECTION_NAME = input('Enter your collection name (default: crew): ') or 'crew'
CACHE_COLLECTION = input('Enter your cache collection name (default: cache): ') or 'cache'

# Check if OpenAI API key is set
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set")

# Connecting to the Couchbase Cluster

Connecting to a Couchbase cluster is the foundation of our project. Couchbase will serve as our primary data store, handling all the storage and retrieval operations required for our semantic search engine.

In [ ]:
try:
    auth = PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)
    options = ClusterOptions(auth)
    cluster = Cluster(CB_HOST, options)
    cluster.wait_until_ready(timedelta(seconds=5))
    logging.info("Successfully connected to Couchbase")
except Exception as e:
    raise ConnectionError(f"Failed to connect to Couchbase: {str(e)}")

# Setting Up Collections in Couchbase

In Couchbase, data is organized in buckets, which can be further divided into scopes and collections. Before we can store any data, we need to ensure that our collections exist.

In [ ]:
def setup_collection(cluster, bucket_name, scope_name, collection_name):
    try:
        bucket = cluster.bucket(bucket_name)
        bucket_manager = bucket.collections()

        # Check if collection exists, create if it doesn't
        collections = bucket_manager.get_all_scopes()
        collection_exists = any(
            scope.name == scope_name and collection_name in [col.name for col in scope.collections]
            for scope in collections
        )

        if not collection_exists:
            logging.info(f"Collection '{collection_name}' does not exist. Creating it...")
            bucket_manager.create_collection(scope_name, collection_name)
            logging.info(f"Collection '{collection_name}' created successfully.")
        else:
            logging.info(f"Collection '{collection_name}' already exists. Skipping creation.")

        collection = bucket.scope(scope_name).collection(collection_name)

        # Ensure primary index exists
        try:
            cluster.query(f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{bucket_name}`.`{scope_name}`.`{collection_name}`").execute()
            logging.info("Primary index present or created successfully.")
        except Exception as e:
            logging.warning(f"Error creating primary index: {str(e)}")

        # Clear all documents in the collection
        try:
            query = f"DELETE FROM `{bucket_name}`.`{scope_name}`.`{collection_name}`"
            cluster.query(query).execute()
            logging.info("All documents cleared from the collection.")
        except Exception as e:
            logging.warning(f"Error while clearing documents: {str(e)}. The collection might be empty.")

        return collection
    except Exception as e:
        raise RuntimeError(f"Error setting up collection: {str(e)}")

setup_collection(cluster, CB_BUCKET_NAME, SCOPE_NAME, COLLECTION_NAME)
setup_collection(cluster, CB_BUCKET_NAME, SCOPE_NAME, CACHE_COLLECTION)

# Loading Couchbase Vector Search Index

Semantic search requires an efficient way to retrieve relevant documents based on a user's query. This is where the Couchbase **Vector Search Index** comes into play. In this step, we load the Vector Search Index definition from a JSON file, which specifies how the index should be structured.

For more information on creating a vector search index, please follow the [instructions](https://docs.couchbase.com/cloud/vector-search/create-vector-search-index-ui.html).

In [ ]:
try:
    with open('crew_index.json', 'r') as file:
        index_definition = json.load(file)
except Exception as e:
    raise ValueError(f"Error loading index definition: {str(e)}")

# Creating or Updating Search Indexes

With the index definition loaded, the next step is to create or update the **Vector Search Index** in Couchbase. This step is crucial because it optimizes our database for vector similarity search operations.

In [ ]:
try:
    scope_index_manager = cluster.bucket(CB_BUCKET_NAME).scope(SCOPE_NAME).search_indexes()

    # Check if index already exists
    existing_indexes = scope_index_manager.get_all_indexes()
    index_name = index_definition["name"]

    if index_name in [index.name for index in existing_indexes]:
        logging.info(f"Index '{index_name}' found")
    else:
        logging.info(f"Creating new index '{index_name}'...")

    # Create SearchIndex object from JSON definition
    search_index = SearchIndex.from_json(index_definition)

    # Upsert the index (create if not exists, update if exists)
    scope_index_manager.upsert_index(search_index)
    logging.info(f"Index '{index_name}' successfully created/updated.")

except QueryIndexAlreadyExistsException:
    logging.info(f"Index '{index_name}' already exists. Skipping creation/update.")

except InternalServerFailureException as e:
    error_message = str(e)
    logging.error(f"InternalServerFailureException raised: {error_message}")

    try:
        # Accessing the response_body attribute from the context
        error_context = e.context
        response_body = error_context.response_body
        if response_body:
            error_details = json.loads(response_body)
            error_message = error_details.get('error', '')

            if "collection: 'crew' doesn't belong to scope: 'shared'" in error_message:
                raise ValueError("Collection 'crew' does not belong to scope 'shared'. Please check the collection and scope names.")

    except ValueError as ve:
        logging.error(str(ve))
        raise

    except Exception as json_error:
        logging.error(f"Failed to parse the error message: {json_error}")
        raise RuntimeError(f"Internal server error while creating/updating search index: {error_message}")

# Load the TREC Dataset

To build a search engine, we need data to search through. We use the TREC dataset, a well-known benchmark in the field of information retrieval.

In [ ]:
try:
    trec = load_dataset('trec', split='train[:1000]')
    logging.info(f"Successfully loaded TREC dataset with {len(trec)} samples")
except Exception as e:
    raise ValueError(f"Error loading TREC dataset: {str(e)}")

# Setting Up OpenAI Embeddings and LLM

We'll use OpenAI's models for embeddings and language generation, which will be used by our CrewAI agents.

In [ ]:
try:
    embeddings = OpenAIEmbeddings(
        openai_api_key=OPENAI_API_KEY,
        model="text-embedding-ada-002"
    )
    
    llm = ChatOpenAI(
        openai_api_key=OPENAI_API_KEY,
        model="gpt-4-turbo-preview",
        temperature=0
    )
    logging.info("Successfully created OpenAI clients")
except Exception as e:
    raise ValueError(f"Error creating OpenAI clients: {str(e)}")

# Setting Up the Couchbase Vector Store

A vector store is where we'll keep our embeddings. Unlike the FTS index, which is used for text-based search, the vector store is specifically designed to handle embeddings and perform similarity searches.

In [ ]:
try:
    vector_store = CouchbaseVectorStore(
        cluster=cluster,
        bucket_name=CB_BUCKET_NAME,
        scope_name=SCOPE_NAME,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        index_name=INDEX_NAME,
    )
    logging.info("Successfully created vector store")
except Exception as e:
    raise ValueError(f"Failed to create vector store: {str(e)}")

# Saving Data to the Vector Store

With the vector store set up, the next step is to populate it with data. We save the TREC dataset to the vector store in batches.

In [ ]:
try:
    batch_size = 50
    logging.disable(sys.maxsize) # Disable logging to prevent tqdm output
    for i in tqdm(range(0, len(trec['text']), batch_size), desc="Processing Batches"):
        batch = trec['text'][i:i + batch_size]
        documents = [Document(page_content=text) for text in batch]
        uuids = [str(uuid4()) for _ in range(len(documents))]
        vector_store.add_documents(documents=documents, ids=uuids)
    logging.disable(logging.NOTSET) # Re-enable logging
except Exception as e:
    raise RuntimeError(f"Failed to save documents to vector store: {str(e)}")

# Setting Up a Couchbase Cache

To further optimize our system, we set up a Couchbase-based cache. A cache is a temporary storage layer that holds data that is frequently accessed, speeding up operations by reducing the need to repeatedly retrieve the same information from the database.

In [ ]:
try:
    cache = CouchbaseCache(
        cluster=cluster,
        bucket_name=CB_BUCKET_NAME,
        scope_name=SCOPE_NAME,
        collection_name=CACHE_COLLECTION,
    )
    logging.info("Successfully created cache")
    set_llm_cache(cache)
except Exception as e:
    raise ValueError(f"Failed to create cache: {str(e)}")

# Creating CrewAI Agents

Now we'll create specialized agents using CrewAI. Each agent will have a specific role in our RAG system:
1. Research Agent - Responsible for retrieving relevant documents
2. Writing Agent - Responsible for generating responses based on retrieved documents

In [ ]:
# Research Agent for document retrieval
researcher = Agent(
    role='Research Expert',
    goal='Find the most relevant documents to answer user queries',
    backstory="""You are an expert researcher with deep knowledge in information retrieval. 
    Your job is to find the most relevant documents to help answer user questions.""",
    tools=[],
    llm=llm,
    verbose=True
)

# Writing Agent for response generation
writer = Agent(
    role='Technical Writer',
    goal='Generate clear and accurate responses based on provided documents',
    backstory="""You are a skilled technical writer who excels at synthesizing information 
    from multiple sources to create clear and accurate responses.""",
    tools=[],
    llm=llm,
    verbose=True
)

logging.info("Successfully created CrewAI agents")

# Creating CrewAI Tasks

Now we'll define the tasks that our agents will perform. These tasks form the workflow of our RAG system.

In [ ]:
def create_tasks(query):
    # Task for retrieving relevant documents
    research_task = Task(
        description=f"Find relevant documents to answer: {query}",
        agent=researcher,
        context=lambda: format_docs(vector_store.similarity_search(query))
    )

    # Task for generating response
    writing_task = Task(
        description="Generate a comprehensive response based on the retrieved documents",
        agent=writer,
        context=lambda: research_task.output
    )

    return [research_task, writing_task]

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

logging.info("Successfully created task definitions")

# Creating and Running the Crew

Now we'll create a Crew that coordinates our agents to perform RAG operations.

In [ ]:
query = "What caused the 1929 Great Depression?"

try:
    # Create crew with tasks
    crew = Crew(
        agents=[researcher, writer],
        tasks=create_tasks(query),
        process=Process.sequential,
        verbose=True
    )

    # Execute the crew's tasks
    start_time = time.time()
    result = crew.kickoff()
    elapsed_time = time.time() - start_time

    print(f"\nCrewAI Response (completed in {elapsed_time:.2f} seconds):")
    print(result)

except Exception as e:
    raise RuntimeError(f"Error executing CrewAI tasks: {str(e)}")

# Testing with Multiple Queries

Let's test our CrewAI-powered RAG system with multiple queries to see how it handles different types of questions.

In [ ]:
try:
    queries = [
        "Why do heavier objects travel downhill faster?",
        "What caused the 1929 Great Depression?", # Repeated query
        "Why do heavier objects travel downhill faster?",  # Repeated query
    ]

    for i, query in enumerate(queries, 1):
        print(f"\nQuery {i}: {query}")
        start_time = time.time()

        # Create and execute crew for each query
        crew = Crew(
            agents=[researcher, writer],
            tasks=create_tasks(query),
            process=Process.sequential,
            verbose=True
        )
        result = crew.kickoff()

        elapsed_time = time.time() - start_time
        print(f"Response: {result}")
        print(f"Time taken: {elapsed_time:.2f} seconds")

except Exception as e:
    raise ValueError(f"Error generating CrewAI responses: {str(e)}")